
Feedback : shins777@gmail.com

이 Colab은 LLM 처리중에 Function을 호출하는 Function call 기능을 설명하는 예제입니다.

#라이브러리 설치
상세한 Python API 정보는 아래 URL 참고하세요.
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [1]:
!pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 751.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 9.9 MB/s eta 0:00:00


#GCP 사용자 인증 / 환경설정

Colab 환경에서는 아래와 같이 인증이 필요합니다.

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_vertexai.llms import VertexAI

본인의 GCP 환경에 맞게 아래 설정을 구성하세요. 단, model_name은 "gemini-pro" 를 그대로 사용하세요.

In [4]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

# 모델 초기화 및 기본 실행
1. Langchain을 기반으로한 기본 Operation
2. Responsible AI
3. Stop word

구글은 아래와 같이 생성형 AI 처리시 Responsible AI 부분으로 Harmcategory 에 따른 LLM 응답 조절이 가능합니다.


Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold



In [5]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

*  https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro

아래 내용은 Langchain 기반에서의 gemini_pro 를 초기화하는 방법입니다.
업무에 따라서 Langchain 을 사용하셔도 좋고, 만일 latency 가 중요하다고 하면, Google 자체 API를 사용하는것도 Latency를 줄이는 방법이 될수 있습니다.

In [6]:
gemini_pro = VertexAI( model_name=model_name,
                  project=project,
                  location=location,
                  verbose=True,
                  streaming=True,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 10
                 )

# Function Call
LLM 처리중에 외부 함수 실행 할때 필요한 기능으로 Function call을 활용할 수 있습니다.

아래 함수는 LLM 처리 과정에서 함수가 호출되면서 처리되는 예제이며, 이 해당 함수의 로직내에서 특정 함수를 호출하거나 또는 외부 REST API를 호출해서 로직을 구성할 수 있습니다.

#### LLM 처리중에 호출 되는 함수 정의

In [7]:
def get_stock_price(ticker):
  print(ticker)
  return "Stock price is 445.55"

#### Gemini Function call 기능 예제

In [8]:
from vertexai.preview.generative_models import FunctionDeclaration
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import Part
from vertexai.preview.generative_models import Tool

def function_call(prompt):
  # Tools
  tools = Tool(function_declarations=[
      FunctionDeclaration(
          name="get_stock_price",
          description="Get the current stock price of a given company",
          parameters={
              "type": "object",
              "properties": {
                  "ticker": {
                      "type": "string",
                      "description": "Stock ticker symbol"
                  }
              }
          },
      )
  ])

  # Model Initialization
  model = GenerativeModel("gemini-pro",
                          generation_config={"temperature": 0},
                          tools=[tools])

  chat = model.start_chat()

  # Send a prompt to the chat
  response = chat.send_message(prompt)

  function_call = response.candidates[0].content.parts[0].function_call

  function_handlers = {
      "get_stock_price": get_stock_price,

  }

  if function_call.name in function_handlers:
      function_name = function_call.name
      chat_response = ""

      args = {key: value for key, value in function_call.args.items()}

      if args:
          function_response = function_handlers[function_name](args)

          part_data = Part.from_function_response(
                  name=function_name,
                  response={
                      "content": function_response,
                  }
          )

          cal_eq = part_data.to_dict()['function_response']['response']['content']

          response = chat.send_message(part_data,)
          chat_response = response.candidates[0].content.parts[0].text

          print(f"Function call 이후 응답: {chat_response}")
          return chat_response
      else:
          print("No arguments found for the function.")
          return "주식정보를 제공해야 할 회사 이름을 알려주세요."
  else:
      print("Function call이 되지 않은 응답")
      return response.text




#### Function call 이 되는 prompt로 호출

In [9]:
prompt = "구글의 주식 가격이 현재 얼마인가요 ?"

function_call(prompt)

{'ticker': 'GOOG'}
Function call 이후 응답: 현재 구글의 주식 가격은 445.55입니다.


'현재 구글의 주식 가격은 445.55입니다.'

#### Function call 이 호출되지 않는 Prompt로 호출

In [10]:
prompt = "오늘의 날씨는 어떤가요 ?"

function_call(prompt)

Function call이 되지 않은 응답


'죄송합니다. 해당 요청을 충족할 수 없습니다. 날씨 정보를 제공하는 도구가 없습니다.'